In [1]:
import os
from dotenv import load_dotenv

from langchain.agents import AgentExecutor, create_openai_tools_agent

load_dotenv()

OPENAI_SECRET_KEY = os.getenv("OPENAI_SECRET_KEY")

Make a custom tool which writes an entry to a diary.
<br>Reference: https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/

In [2]:
import datetime
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Optional

In [3]:
class DiaryEntry:

    def __init__(self, date: datetime.date, text: str):
        self.date = date
        self.text = text

    def __str__(self):
        return self.text

class Diary:

    def __init__(self):

        self.entries = {} # {datetime: [entry, ..]}

    def write_entry(self, entry):

        self.entries[entry.date] = self.entries.get(entry.date, []) + [entry]

    def get_entries(self, date):

        return self.entries.get(date, [])

In [4]:
test_diary = Diary()
test_diary.write_entry(
    DiaryEntry(date = datetime.date(2024,6,6), text = "I ate a lot")
)
print(test_diary.get_entries(datetime.date(2024,6,6))[0])

I ate a lot


In [5]:
class WriteToDiaryInput(BaseModel):
    date: Optional[datetime.date] = Field(description = "The date in the diary wherein the text will be written. Defaults to today if not provided.")
    text: str = Field(description = "The text to be written into the diary")
    
DIARY = Diary()

@tool("write-to-diary-tool", args_schema = WriteToDiaryInput)
def write_to_diary(text: str, date: datetime.date = datetime.date.today()):
    """
    Writes the text into the diary in the page of the corresponding date.
    Writes to the page for today if date is not provided.
    """
    DIARY.write_entry(
        DiaryEntry(
            date = date,
            text = text
        )
    )
    return True

In [6]:
print(write_to_diary.name)
print(write_to_diary.description)
print(write_to_diary.args)

write-to-diary-tool
Writes the text into the diary in the page of the corresponding date.
Writes to the page for today if date is not provided.
{'date': {'title': 'Date', 'description': 'The date in the diary wherein the text will be written. Defaults to today if not provided.', 'type': 'string', 'format': 'date'}, 'text': {'title': 'Text', 'description': 'The text to be written into the diary', 'type': 'string'}}


Make a conversational agent that can use the tool to write to the diary when commanded

In [7]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage

In [8]:
llm = ChatOpenAI(
    model = "gpt-4o",
    temperature = 0,
    max_tokens = None,
    timeout = 300,
    max_retries = 2,
    openai_api_key = OPENAI_SECRET_KEY,
)

In [9]:
tools = [write_to_diary]

In [10]:
system_prompt = '''
You are a helpful assistant.
When asked to write something to the human's diary, use the write-to-diary-tool.
Before writing anything to the diary, confirm with the human whether he wants it written.
'''.strip()

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)

In [12]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
output = agent_executor.invoke({"input": "i had a tough day today."})
print(type(output))
print(output)



> Entering new AgentExecutor chain...
I'm sorry to hear that. Would you like to write about it in your diary?

> Finished chain.
<class 'dict'>
{'input': 'i had a tough day today.', 'output': "I'm sorry to hear that. Would you like to write about it in your diary?"}


In [14]:
output = agent_executor.invoke({
    "chat_history": [
        HumanMessage(content="i had a tough day today."),
        AIMessage(content="I'm sorry to hear that you had a tough day. Would you like me to write about it in your diary? If so, please provide some details you'd like to include."),
    ],
    "input": "I was stuck in traffic for 2 hours. yes, please write this in my diary.",
})
print(type(output))
print(output)



> Entering new AgentExecutor chain...

Invoking: `write-to-diary-tool` with `{'text': 'I had a tough day today. I was stuck in traffic for 2 hours.'}`


TrueI've written about your tough day and the traffic situation in your diary. If there's anything else you'd like to add or talk about, feel free to let me know.

> Finished chain.
<class 'dict'>
{'chat_history': [HumanMessage(content='i had a tough day today.'), AIMessage(content="I'm sorry to hear that you had a tough day. Would you like me to write about it in your diary? If so, please provide some details you'd like to include.")], 'input': 'I was stuck in traffic for 2 hours. yes, please write this in my diary.', 'output': "I've written about your tough day and the traffic situation in your diary. If there's anything else you'd like to add or talk about, feel free to let me know."}


In [16]:
print(DIARY.get_entries(datetime.date.today())[0])

I had a tough day today. I was stuck in traffic for 2 hours.
